In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time

In [2]:
response = requests.get('https://arxiv.org/list/cs/recent')
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
author = []
title = []
subject = []
abstract = []
web = []
for i in range(0, 76, 25):
    url = f'https://arxiv.org/list/cs/pastweek?skip={i}&show=25'
    r = requests.get(url)
    r.raise_for_status()
    soup = BeautifulSoup(r.text)
    for i in soup.select("#dlpage > dl > dd > div > div.list-title.mathjax "):
        title.append(i.get_text().replace('Title: ', "").replace('\n', ''))
    for i in soup.select("#dlpage > dl > dd > div > div.list-authors"):
        author.append(i.get_text().replace('Authors:', "").replace('\n', ''))
    for i in soup.select("#dlpage > dl > dd > div > div.list-subjects"):
        subject.append(i.get_text().replace("Subjects:", "").replace('\n', ''))
    for i in soup.select('#dlpage > dl > dt > span > a:nth-child(1)'):
        web.append('https://arxiv.org'+ i.attrs['href'])
    time.sleep(1)

In [27]:
# 新增代码，爬取每篇论文的摘要
for link in web:
    # 发起请求获取论文的详细页面
    detailed_response = requests.get(link,timeout=7)
    detailed_response.raise_for_status()
    
    # 解析详细页面
    detailed_soup = BeautifulSoup(detailed_response.text, 'html.parser')
    for i in detailed_soup.select("#abs > blockquote"):
        abstract.append(i.get_text().replace('Abstract:', "").replace('\n', ''))
    time.sleep(1)

SSLError: HTTPSConnectionPool(host='arxiv.org', port=443): Max retries exceeded with url: /abs/2403.00745 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))

In [28]:
with open('arxiv_papers.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入列名
    writer.writerow(['Title', 'Authors', 'Subjects', 'Abstract'])
    # 写入论文信息
    for i in range(len(abstract)):
        writer.writerow([title[i], author[i], subject[i], abstract[i]])

print('CSV文件写入完成。')

CSV文件写入完成。
